In [2]:
!pip install yfinance --quiet




In [3]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
from typing import Dict, Any, Iterable

In [4]:
tickers = ["AAPL"]
start = "2015-01-01"
end = "2025-10-30"

out_dir = "/content/data"
os.makedirs(out_dir, exist_ok=True)

In [5]:
all_dfs = []
for t in tickers:
    tick = t
    print(f"Downloading {tick} ...")
    df = yf.download(tick, start=start, end=end, progress=False, auto_adjust=False)

    #  столбцы — MultiIndex, пытаемся раскрыть нужный уровень
    if isinstance(df.columns, pd.MultiIndex):
        #  верхний уровень = тикер, нижний = ['Open','High',...]
        if df.columns.nlevels == 2:
            lvl0 = df.columns.get_level_values(0)
            lvl1 = df.columns.get_level_values(1)
            expected = ["Open", "High", "Low", "Close", "Volume", "Adj Close"]
            if set(expected).issubset(set(lvl1)):
                df.columns = lvl1
            elif set(expected).issubset(set(lvl0)):
                df.columns = lvl0
            else:
                df.columns = ['_'.join(map(str, c)).strip() for c in df.columns.values]
        else:
            df.columns = ['_'.join(map(str, c)).strip() for c in df.columns.values]

    df = df.reset_index().rename(columns={
        "Date": "date",
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Adj Close": "adj_close",
        "Volume": "volume"
    })

    # Если adj_close отсутствует — копируем close
    if "adj_close" not in df.columns:
        df["adj_close"] = df["close"]

    df["ticker"] = tick
    df = df.sort_values("date").reset_index(drop=True)

    df["prev_close"] = df["adj_close"].shift(1)
    df["return"] = df["adj_close"].pct_change()
    df["log_return"] = np.log(df["adj_close"] / df["prev_close"])
    df["hl_spread"] = df["high"] - df["low"]
    df["oc_spread"] = (df["close"] - df["open"]) / df["open"]
    df["typical_price"] = (df["high"] + df["low"] + df["close"]) / 3
    df["dayofweek"] = df["date"].dt.dayofweek
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    df["is_month_start"] = df["date"].dt.is_month_start.astype(int)
    df["next_return"] = df["close"].shift(-1) / df["close"] - 1

    keep_cols = [
        "date", "ticker", "open", "high", "low", "close", "adj_close", "volume",
        "prev_close", "return", "log_return", "hl_spread", "oc_spread",
        "typical_price", "dayofweek", "month", "year", "is_month_start",
        "next_return"
    ]
    keep_existing = [c for c in keep_cols if c in df.columns]
    df = df[keep_existing]
    df = df.dropna(subset=["open", "high", "low", "close"], how="any")

    all_dfs.append(df)

# Конкатенируем только если есть данные
if len(all_dfs) == 0:
    raise RuntimeError("No valid dataframes collected (all_dfs is empty). Check the diagnostic messages above.")
prices = pd.concat(all_dfs, ignore_index=True)
prices = prices.sort_values(["ticker", "date"]).reset_index(drop=True)

In [6]:
prices.to_csv("/content/data/prices.csv", index=False)

In [7]:
# Вывод какой-то статистики и превью данных

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

print("=== preview (first 20 rows) ===")
display(prices.head(20))

print("\n=== info ===")
print(prices.info())

print("\n=== basic describe (numeric cols) ===")
display(prices.describe().T)

print("\n=== nulls per column ===")
print(prices.isnull().sum())

print("\n=== dtypes ===")
print(prices.dtypes)

# Ensure date is datetime
if not pd.api.types.is_datetime64_any_dtype(prices['date']):
    print("\nConverting prices['date'] to datetime...")
    prices['date'] = pd.to_datetime(prices['date'])
    print("Converted.")

print("\n=== summary by ticker: date range, rows, missing values ===")
summary = prices.groupby("ticker").agg(
    date_min = ("date","min"),
    date_max = ("date","max"),
    rows = ("date","count"),
    n_null_adj_close = ("adj_close", lambda s: s.isnull().sum()),
    n_null_volume = ("volume", lambda s: s.isnull().sum()),
)
display(summary.sort_values(["rows"], ascending=False))

print("\n=== duplicate dates per ticker (should be 0) ===")
dups = prices.groupby("ticker").apply(lambda g: g['date'].duplicated().sum()).rename("n_duplicate_dates")
display(dups.sort_values(ascending=False).head(20))

print("\n=== sample rows for each ticker (last 3 rows) ===")
for t, g in prices.groupby("ticker"):
    print(f"\n--- {t} ({len(g)} rows) last 3 rows ---")
    display(g.sort_values("date").tail(3))

print("\n=== quick consistency checks ===")
# prev_close vs adj_close.shift(1) per ticker
def check_prev_close_consistency(g):
    # compare prev_close column with adj_close shifted within the same ticker
    if 'prev_close' not in g.columns:
        return pd.Series({"prev_close_missing": True})
    s = (g['prev_close'] != g['adj_close'].shift(1)) & ~(g['prev_close'].isnull() & g['adj_close'].shift(1).isnull())
    return pd.Series({
        "n_prev_close_mismatch": int(s.sum()),
        "pct_prev_close_mismatch": float(s.sum() / max(1, len(g)))
    })

consistency = prices.groupby("ticker").apply(check_prev_close_consistency)
display(consistency.sort_values("n_prev_close_mismatch", ascending=False).head(20))

print("\n=== end checks ===")

=== preview (first 20 rows) ===


Price,date,ticker,open,high,low,close,adj_close,volume,prev_close,return,log_return,hl_spread,oc_spread,typical_price,dayofweek,month,year,is_month_start,next_return
0,2015-01-02,AAPL,27.847500,27.860001,26.837500,27.332500,24.261042,212818400,NaN,NaN,NaN,1.022501,-0.018494,27.343334,4,1,2015,0,-0.028172
1,2015-01-05,AAPL,27.072500,27.162500,26.352501,26.562500,23.577574,257142000,24.261042,-0.028171,-0.028576,0.809999,-0.018838,26.692500,0,1,2015,0,0.000094
2,2015-01-06,AAPL,26.635000,26.857500,26.157499,26.565001,23.579798,263188400,23.577574,0.000094,0.000094,0.700001,-0.002628,26.526667,1,1,2015,0,0.014022
3,2015-01-07,AAPL,26.799999,27.049999,26.674999,26.937500,23.910431,160423600,23.579798,0.014022,0.013924,0.375000,0.005131,26.887499,2,1,2015,0,0.038422
4,2015-01-08,AAPL,27.307501,28.037500,27.174999,27.972500,24.829124,237458000,23.910431,0.038422,0.037703,0.862501,0.024352,27.728333,3,1,2015,0,0.001073
5,2015-01-09,AAPL,28.167500,28.312500,27.552500,28.002501,24.855755,214798000,24.829124,0.001073,0.001072,0.760000,-0.005858,27.955833,4,1,2015,0,-0.024641
6,2015-01-12,AAPL,28.150000,28.157499,27.200001,27.312500,24.243298,198603200,24.855755,-0.024640,-0.024949,0.957499,-0.029751,27.556667,0,1,2015,0,0.008879
7,2015-01-13,AAPL,27.857500,28.200001,27.227501,27.555000,24.458544,268367600,24.243298,0.008879,0.008839,0.972500,-0.010859,27.660834,1,1,2015,0,-0.003811
8,2015-01-14,AAPL,27.260000,27.622499,27.125000,27.450001,24.365341,195826400,24.458544,-0.003811,-0.003818,0.497499,0.006970,27.399167,2,1,2015,0,-0.027140
9,2015-01-15,AAPL,27.500000,27.514999,26.665001,26.705000,23.704063,240056000,24.365341,-0.027140,-0.027515,0.849998,-0.028909,26.961667,3,1,2015,0,-0.007770



=== info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2723 entries, 0 to 2722
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            2723 non-null   datetime64[ns]
 1   ticker          2723 non-null   object        
 2   open            2723 non-null   float64       
 3   high            2723 non-null   float64       
 4   low             2723 non-null   float64       
 5   close           2723 non-null   float64       
 6   adj_close       2723 non-null   float64       
 7   volume          2723 non-null   int64         
 8   prev_close      2722 non-null   float64       
 9   return          2722 non-null   float64       
 10  log_return      2722 non-null   float64       
 11  hl_spread       2723 non-null   float64       
 12  oc_spread       2723 non-null   float64       
 13  typical_price   2723 non-null   float64       
 14  dayofweek       2723 non-null   int32     

,count,mean,min,25%,50%,75%,max,std
Price,,,,,,,,
date,2723,2020-05-30 07:06:45.875872256,2015-01-02 00:00:00,2017-09-14 12:00:00,2020-06-01 00:00:00,2023-02-11 12:00:00,2025-10-29 00:00:00,NaN
open,2723.0,105.935044,22.5,38.9725,81.047501,169.010002,269.279999,71.342559
high,2723.0,107.087566,22.9175,39.286251,81.550003,170.559998,271.410004,72.11429
low,2723.0,104.868596,22.3675,38.59,80.3675,167.5,268.149994,70.644328
close,2723.0,106.03375,22.584999,38.9625,81.237503,168.830002,269.700012,71.427126
adj_close,2723.0,103.778014,20.624054,36.465322,78.646622,166.586006,269.700012,71.900294
volume,2723.0,112439386.705839,23234700.0,65927800.0,95174000.0,137677600.0,648825200.0,68021360.832596
prev_close,2722.0,103.717058,20.624054,36.460015,78.594582,166.555893,269.0,71.843099
return,2722.0,0.001052,-0.128647,-0.007395,0.000994,0.010099,0.153289,0.018284



=== nulls per column ===
Price
date              0
ticker            0
open              0
high              0
low               0
close             0
adj_close         0
volume            0
prev_close        1
return            1
log_return        1
hl_spread         0
oc_spread         0
typical_price     0
dayofweek         0
month             0
year              0
is_month_start    0
next_return       1
dtype: int64

=== dtypes ===
Price
date              datetime64[ns]
ticker                    object
open                     float64
high                     float64
low                      float64
close                    float64
adj_close                float64
volume                     int64
prev_close               float64
return                   float64
log_return               float64
hl_spread                float64
oc_spread                float64
typical_price            float64
dayofweek                  int32
month                      int32
year                     

...

Price,date,ticker,open,high,low,close,adj_close,volume,prev_close,return,log_return,hl_spread,oc_spread,typical_price,dayofweek,month,year,is_month_start,next_return
2720,2025-10-27,AAPL,264.880005,269.119995,264.649994,268.809998,268.809998,44888200,262.820007,0.022791,0.022535,4.470001,0.014837,267.526662,0,10,2025,0,0.000707
2721,2025-10-28,AAPL,268.989990,269.890015,268.149994,269.000000,269.000000,41534800,268.809998,0.000707,0.000707,1.740021,0.000037,269.013336,1,10,2025,0,0.002602
2722,2025-10-29,AAPL,269.279999,271.410004,267.109985,269.700012,269.700012,51086700,269.000000,0.002602,0.002599,4.300018,0.001560,269.406667,2,10,2025,0,NaN



=== quick consistency checks ===


/tmp/ipython-input-1649437742.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  consistency = prices.groupby("ticker").apply(check_prev_close_consistency)


,n_prev_close_mismatch,pct_prev_close_mismatch
ticker,,
AAPL,0.0,0.0



=== end checks ===


# Шаг 2
генерация фичей

In [8]:
def rsi(series: pd.Series, window: int = 14) -> pd.Series:
  delta=series.diff()
  gain=delta.clip(lower=0.0)
  loss=-delta.clip(upper=0.0)
  avg_gain=gain.ewm(alpha=1.0/window,adjust=False).mean()
  avg_loss = loss.ewm(alpha=1.0/window, adjust=False).mean()
  rs= avg_gain / (avg_loss.replace(0, np.nan))
  rsi = 100 - (100 / (1 + rs))
  rsi = rsi.fillna(100.0 * (avg_loss == 0).astype(float))
  return rsi




In [9]:
LAGS = (1, 2, 3)
MA_WINDOWS = (5, 20)
RSI_WINDOW = 14
VOL_WINDOW = 10
MOMENTUM_WINDOW = 5

In [10]:
df = pd.read_csv("/content/data/prices.csv", parse_dates=["date"])

price_col = "adj_close"

df=df.sort_values(["date"]).reset_index(drop=True)
g=df.groupby("ticker",group_keys=False)
for lag in LAGS:
  df[f"r_lag{lag}"]=g["return"].shift(lag)

# moving averages
for w in MA_WINDOWS:
    df[f"ma{w}"] = g[price_col].transform(lambda s, ww=w: s.rolling(window=ww, min_periods=1).mean())

df[f"rsi{RSI_WINDOW}"] = g[price_col].transform(lambda s: rsi(s, window=RSI_WINDOW))
df[f"vol_{VOL_WINDOW}"] = g["return"].transform(lambda s: s.rolling(window=VOL_WINDOW, min_periods=1).std())
df[f"momentum_{MOMENTUM_WINDOW}"] = g[price_col].transform(lambda s: s / s.shift(MOMENTUM_WINDOW) - 1)
#volume fetures
df["vol_change"] = g["volume"].pct_change()
df["vol_roll_mean_20"] = g["volume"].transform(lambda s: s.rolling(window=20, min_periods=1).mean())
df["vol_roll_std_20"] = g["volume"].transform(lambda s: s.rolling(window=20, min_periods=1).std())

df["target"] = np.nan

df.to_csv("/content/data/features.csv", index=False)


In [11]:
display(df.head(12))

,date,ticker,open,high,low,close,adj_close,volume,prev_close,return,log_return,hl_spread,oc_spread,typical_price,dayofweek,month,year,is_month_start,next_return,r_lag1,r_lag2,r_lag3,ma5,ma20,rsi14,vol_10,momentum_5,vol_change,vol_roll_mean_20,vol_roll_std_20,target
0,2015-01-02,AAPL,27.847500,27.860001,26.837500,27.332500,24.261042,212818400,NaN,NaN,NaN,1.022501,-0.018494,27.343334,4,1,2015,0,-0.028172,NaN,NaN,NaN,24.261042,24.261042,0.000000,NaN,NaN,NaN,2.128184e+08,NaN,NaN
1,2015-01-05,AAPL,27.072500,27.162500,26.352501,26.562500,23.577574,257142000,24.261042,-0.028171,-0.028576,0.809999,-0.018838,26.692500,0,1,2015,0,0.000094,NaN,NaN,NaN,23.919308,23.919308,0.000000,NaN,NaN,0.208270,2.349802e+08,3.134152e+07,NaN
2,2015-01-06,AAPL,26.635000,26.857500,26.157499,26.565001,23.579798,263188400,23.577574,0.000094,0.000094,0.700001,-0.002628,26.526667,1,1,2015,0,0.014022,-0.028171,NaN,NaN,23.806138,23.806138,0.025024,0.019987,NaN,0.023514,2.443829e+08,2.750236e+07,NaN
3,2015-01-07,AAPL,26.799999,27.049999,26.674999,26.937500,23.910431,160423600,23.579798,0.014022,0.013924,0.375000,0.005131,26.887499,2,1,2015,0,0.038422,0.000094,-0.028171,NaN,23.832211,23.832211,3.876187,0.021499,NaN,-0.390461,2.233931e+08,4.760825e+07,NaN
4,2015-01-08,AAPL,27.307501,28.037500,27.174999,27.972500,24.829124,237458000,23.910431,0.038422,0.037703,0.862501,0.024352,27.728333,3,1,2015,0,0.001073,0.014022,0.000094,-0.028171,24.031594,24.031594,13.811035,0.027797,NaN,0.480194,2.262061e+08,4.170699e+07,NaN
5,2015-01-09,AAPL,28.167500,28.312500,27.552500,28.002501,24.855755,214798000,24.829124,0.001073,0.001072,0.760000,-0.005858,27.955833,4,1,2015,0,-0.024641,0.038422,0.014022,0.000094,24.150536,24.168954,14.088224,0.024178,0.024513,-0.095427,2.243047e+08,3.759348e+07,NaN
6,2015-01-12,AAPL,28.150000,28.157499,27.200001,27.312500,24.243298,198603200,24.855755,-0.024640,-0.024949,0.957499,-0.029751,27.556667,0,1,2015,0,0.008879,0.001073,0.038422,0.014022,24.283681,24.179574,13.048836,0.024798,0.028235,-0.075395,2.206331e+08,3.566639e+07,NaN
7,2015-01-13,AAPL,27.857500,28.200001,27.227501,27.555000,24.458544,268367600,24.243298,0.008879,0.008839,0.972500,-0.010859,27.660834,1,1,2015,0,-0.003811,-0.024640,0.001073,0.038422,24.459430,24.214446,15.410839,0.022877,0.037267,0.351275,2.265999e+08,3.708352e+07,NaN
8,2015-01-14,AAPL,27.260000,27.622499,27.125000,27.450001,24.365341,195826400,24.458544,-0.003811,-0.003818,0.497499,0.006970,27.399167,2,1,2015,0,-0.027140,0.008879,-0.024640,0.001073,24.550412,24.231212,15.218068,0.021260,0.019026,-0.270305,2.231806e+08,3.617336e+07,NaN
9,2015-01-15,AAPL,27.500000,27.514999,26.665001,26.705000,23.704063,240056000,24.365341,-0.027140,-0.027515,0.849998,-0.028909,26.961667,3,1,2015,0,-0.007770,-0.003811,0.008879,-0.024640,24.325400,24.178497,13.890452,0.021950,-0.045312,0.225861,2.248682e+08,3.451955e+07,NaN


# Шаг 3
делаем таргет

In [12]:
df.describe()

,date,open,high,low,close,adj_close,volume,prev_close,return,log_return,hl_spread,oc_spread,typical_price,dayofweek,month,year,is_month_start,next_return,r_lag1,r_lag2,r_lag3,ma5,ma20,rsi14,vol_10,momentum_5,vol_change,vol_roll_mean_20,vol_roll_std_20,target
count,2723,2723.000000,2723.000000,2723.000000,2723.000000,2723.000000,2.723000e+03,2722.000000,2722.000000,2722.000000,2723.000000,2723.000000,2723.000000,2723.000000,2723.000000,2723.000000,2723.000000,2722.000000,2721.000000,2720.000000,2719.000000,2723.000000,2723.000000,2723.000000,2721.000000,2718.000000,2722.000000,2.723000e+03,2.722000e+03,0.0
mean,2020-05-30 07:06:45.875872256,105.935044,107.087566,104.868596,106.033750,103.778014,1.124394e+08,103.717058,0.001052,0.000885,2.218970,0.000857,105.996637,2.023871,6.469703,2019.916269,0.031216,0.001008,0.001052,0.001052,0.001044,103.598360,102.956086,54.762521,0.016107,0.005179,0.049917,1.130737e+08,3.679651e+07,NaN
min,2015-01-02 00:00:00,22.500000,22.917500,22.367500,22.584999,20.624054,2.323470e+07,20.624054,-0.128647,-0.137708,0.144999,-0.076486,22.682500,0.000000,1.000000,2015.000000,0.000000,-0.128647,-0.128647,-0.128647,-0.128647,20.983843,21.427351,0.000000,0.003596,-0.227474,-0.830093,3.780050e+07,5.655687e+06,NaN
25%,2017-09-14 12:00:00,38.972500,39.286251,38.590000,38.962500,36.465322,6.592780e+07,36.460015,-0.007395,-0.007422,0.590000,-0.006406,38.921250,1.000000,4.000000,2017.000000,0.000000,-0.007449,-0.007395,-0.007396,-0.007397,36.516271,36.442803,45.011609,0.010381,-0.016771,-0.175272,7.093051e+07,1.986220e+07,NaN
50%,2020-06-01 00:00:00,81.047501,81.550003,80.367500,81.237503,78.646622,9.517400e+07,78.594582,0.000994,0.000994,1.639999,0.000898,81.135000,2.000000,6.000000,2020.000000,0.000000,0.000934,0.000988,0.000994,0.000988,78.190523,77.186195,55.286454,0.014186,0.006578,-0.017989,1.026992e+08,3.012690e+07,NaN
75%,2023-02-11 12:00:00,169.010002,170.559998,167.500000,168.830002,166.586006,1.376776e+08,166.555893,0.010099,0.010048,3.264996,0.008532,168.806664,3.000000,9.000000,2023.000000,0.000000,0.010054,0.010108,0.010111,0.010090,166.512784,165.661299,64.454089,0.019699,0.027404,0.197954,1.378378e+08,4.815634e+07,NaN
max,2025-10-29 00:00:00,269.279999,271.410004,268.149994,269.700012,269.700012,6.488252e+08,269.000000,0.153289,0.142617,28.720001,0.156441,269.406667,4.000000,12.000000,2025.000000,1.000000,0.153289,0.153289,0.153289,0.153289,265.982001,257.178500,90.695354,0.080056,0.184141,3.771993,3.102953e+08,1.509430e+08,NaN
std,NaN,71.342559,72.114290,70.644328,71.427126,71.900294,6.802136e+07,71.843099,0.018284,0.018267,2.116306,0.014270,71.390934,1.398730,3.396320,3.122582,0.173932,0.018290,0.018287,0.018291,0.018289,71.764871,71.347677,13.280174,0.008981,0.038599,0.357141,5.344238e+07,2.338270e+07,NaN


In [56]:
# target: 1 если next_return > 0 иначе 0
df["target"] = (df["next_return"] > 0).astype(int)

In [57]:
df_clean = df.dropna(subset=["next_return"]).reset_index(drop=True)
df_clean.to_csv("/content/data/dataset.csv", index=False)

# шаг 4
сплит данных

In [58]:
TRAIN_END = "2020-12-31"   # включительно в train
VAL_END   = "2022-12-31"   # включительно в val (val диапазон 2021-01-01 .. 2022-12-31)
TEST_START = "2023-01-01"  # тест с 2023-01-01 и дальше
train_end_dt = pd.to_datetime(TRAIN_END)
val_end_dt = pd.to_datetime(VAL_END)
test_start_dt = pd.to_datetime(TEST_START)
np.random.seed(42)

In [59]:
train_df=df_clean.loc[df_clean["date"]<=train_end_dt].reset_index(drop=True)
val_df=df_clean.loc[(df_clean['date'] > train_end_dt) & (df_clean['date'] <= val_end_dt)].reset_index(drop=True)
test_df=df_clean.loc[df_clean["date"]>=test_start_dt].reset_index(drop=True)

In [60]:
train_path = os.path.join("/content/data", "train.csv")
val_path = os.path.join("/content/data", "val.csv")
test_path = os.path.join("/content/data", "test.csv")
train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)


In [61]:
num_features=[c for c in df_clean.select_dtypes(include=[np.number]).columns.tolist() if c not in {"target", "next_return"}]
num_features

['open',
 'high',
 'low',
 'close',
 'adj_close',
 'volume',
 'prev_close',
 'return',
 'log_return',
 'hl_spread',
 'oc_spread',
 'typical_price',
 'dayofweek',
 'month',
 'year',
 'is_month_start',
 'r_lag1',
 'r_lag2',
 'r_lag3',
 'ma5',
 'ma20',
 'rsi14',
 'vol_10',
 'momentum_5',
 'vol_change',
 'vol_roll_mean_20',
 'vol_roll_std_20']

In [62]:
feat_file = os.path.join("/content/data", "recommended_features.txt")
with open(feat_file, "w", encoding="utf-8") as f:
    for c in num_features:
        f.write(c + "\n")

# Шаг 5
обучаем модели

In [20]:
!pip install lightgbm xgboost joblib --quiet


In [43]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 7.1 MB/s eta 0:00:00


In [98]:
import optuna
from pathlib import Path
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.utils import check_random_state
import joblib
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBClassifier
os.makedirs("/content/models", exist_ok=True)

In [73]:
train=train_df.copy()
val =val_df.copy()
test= test_df.copy()
features=[c for c in num_features if c not in {"date", "ticker", "open", "high", "low", "close", "adj_close", "prev_close", "typical_price", 'ma20','ma5'}] # лучше убрать явные признаки , так как модель будет сильно обращать внимание именно на цену , что будет мешать учится
print("Number of features:", len(features))

Number of features: 18


In [74]:
print(f"Доля единиц в train в столбце target: {train["target"].mean()}")

Доля единиц в train в столбце target: 0.5314361350099271


обучающая выборка не предвзята

In [86]:
X_train, y_train = train[features], train["target"]
X_val, y_val     = val[features], val["target"]
X_test, y_test   = test[features], test["target"]
X_full = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)
y_full = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)
tscv = TimeSeriesSplit(n_splits=5)

**LightGBM**

In [91]:
def objective_lgb_ts(trial):
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state":42 ,
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 200),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.5),
    }
    aucs = []
    best_iters = []
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_full)):
        X_tr, X_va = X_full.iloc[train_idx], X_full.iloc[val_idx]
        y_tr, y_va = y_full.iloc[train_idx], y_full.iloc[val_idx]

        model = lgb.LGBMClassifier(**param, n_estimators=2000)
        callbacks = [
        lgb.early_stopping(stopping_rounds=10),
        lgb.log_evaluation(period=0)  ]

        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="auc",
            callbacks=callbacks,
        )

        y_pred = model.predict_proba(X_va)[:, 1]
        aucs.append(roc_auc_score(y_va, y_pred))
        try:
            best_iters.append(int(model.best_iteration_))
        except Exception:
            pass

    mean_auc = float(np.mean(aucs))
    trial.set_user_attr("best_it_median", int(np.median(best_iters)))
    return mean_auc





In [92]:

study_lgb = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study_lgb.optimize(objective_lgb_ts, n_trials=200, show_progress_bar=True)

[I 2025-11-05 09:53:37,840] A new study created in memory with name: no-name-de88096b-ef02-498d-8d6e-bb45f960d1d9


  0%|          | 0/200 [00:00<?, ?it/s]

/tmp/ipython-input-3439550314.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
/tmp/ipython-input-3439550314.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
/tmp/ipython-input-3439550314.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.5),
/tmp/ipython-input-3439550314.py:12: FutureWarning: suggest_logunif

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[7]	valid_0's auc: 0.509626
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.549393
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.507452
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[13]	valid_0's auc: 0.483811
[I 2025-11-05 09:53:37,967] Trial 0 finished with value: 0.5100562993683888 and parameters: {'num_leaves': 106, 'min_child_samples': 191, 'subsample': 0.839196365086843, 'colsample_bytree': 0.759195090518222, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07, 'learning_rate': 0.001434715951720141}. Best is trial 0 with value: 0.5100562993683888.
Training unt

/tmp/ipython-input-3439550314.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
/tmp/ipython-input-3439550314.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
/tmp/ipython-input-3439550314.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.5),


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's auc: 0.581244
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	valid_0's auc: 0.553084
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.549393
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.552826
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's auc: 0.5202
[I 2025-11-05 09:54:11,727] Trial 196 finished with value: 0.5513494179485269 and parameters: {'num_leaves': 107, 'min_child_samples': 115, 'subsample': 0.5031501058441992, 'colsample_bytree': 0.601021218817227, 'reg_alpha': 0.014452868862704325, 'reg_lambda': 1.0463702494958984e-08, 'learning_rate': 0.17653118710891436}. Best is trial 160 with value: 0.5604869220201849.
Training

/tmp/ipython-input-3439550314.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
/tmp/ipython-input-3439550314.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
/tmp/ipython-input-3439550314.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.5),


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.55069
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's auc: 0.522393
[I 2025-11-05 09:54:11,880] Trial 197 finished with value: 0.5551112541747594 and parameters: {'num_leaves': 102, 'min_child_samples': 111, 'subsample': 0.5693020377816645, 'colsample_bytree': 0.5573086341140157, 'reg_alpha': 0.006889257260664514, 'reg_lambda': 5.155873663589192e-08, 'learning_rate': 0.11878700270002668}. Best is trial 160 with value: 0.5604869220201849.
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[11]	valid_0's auc: 0.571669
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	valid_0's auc: 0.525936
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[6]	valid_0's auc: 0.532555
Train

/tmp/ipython-input-3439550314.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
/tmp/ipython-input-3439550314.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
/tmp/ipython-input-3439550314.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.5),


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's auc: 0.573547
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[13]	valid_0's auc: 0.570303
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's auc: 0.574541
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.545213
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's auc: 0.524729
[I 2025-11-05 09:54:12,224] Trial 199 finished with value: 0.5576667162377349 and parameters: {'num_leaves': 96, 'min_child_samples': 106, 'subsample': 0.6282690478468004, 'colsample_bytree': 0.6137466494589455, 'reg_alpha': 0.005567657515833916, 'reg_lambda': 2.5426230883173245e-08, 'learning_rate': 0.19217457837360605}. Best is trial 160 with value: 0.5604869220201849.


In [93]:
print("LGBM best params:", study_lgb.best_trial.params)
print("LGBM best  AUC:", study_lgb.best_value)
best_it_lgb = study_lgb.best_trial.user_attrs.get("best_it_median", None)
print("LGBM median best_it per fold:", best_it_lgb)

# тренируем на лучших параметрах
best_params_lgb = study_lgb.best_trial.params.copy()
final_n_estimators_lgb = int(best_it_lgb) if best_it_lgb is not None else 200
final_lgb = lgb.LGBMClassifier(**best_params_lgb, n_estimators=final_n_estimators_lgb)
final_lgb.fit(X_full, y_full)

lgb_probs_test = final_lgb.predict_proba(X_test)[:, 1]
lgb_auc_test = roc_auc_score(y_test, lgb_probs_test)
print(f"Final LGBM test AUC: {lgb_auc_test:.4f}")
joblib.dump(final_lgb, "models/lgbm_ts_cv_best.pkl")

LGBM best params: {'num_leaves': 120, 'min_child_samples': 111, 'subsample': 0.5745131333064932, 'colsample_bytree': 0.6070519148944908, 'reg_alpha': 0.0033982961253754486, 'reg_lambda': 3.6640396387940853e-07, 'learning_rate': 0.1160931664590444}
LGBM best  AUC: 0.5604869220201849
LGBM median best_it per fold: 5
Final LGBM test AUC: 0.4785


['models/lgbm_ts_cv_best.pkl']

Модель практически не обучилась , 0.47 - это хуже рандома

In [107]:

def objective_xgb_ts(trial):
    param = {
        "n_estimators": 2000,
        "booster": "gbtree",
        "use_label_encoder": False,
        "eval_metric": "auc",
        "random_state": 42,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "n_jobs": -1,
    }

    aucs = []
    best_iters = []
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_full)):
        X_tr, X_va = X_full.iloc[train_idx], X_full.iloc[val_idx]
        y_tr, y_va = y_full.iloc[train_idx], y_full.iloc[val_idx]

        model = XGBClassifier(**param)

        model.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False)

        y_pred = model.predict_proba(X_va)[:, 1]
        aucs.append(roc_auc_score(y_va, y_pred))
        try:
            best_iters.append(int(model.get_booster().best_iteration))
        except Exception:
            pass

    mean_auc = float(np.mean(aucs))
    if len(best_iters) > 0:
        trial.set_user_attr("best_it_median", int(np.median(best_iters)))
    return mean_auc




study_xgb = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study_xgb.optimize(objective_xgb_ts, n_trials=30, show_progress_bar=True)

print("XGB best params:", study_xgb.best_trial.params)
print("XGB best CV AUC:", study_xgb.best_value)
best_it_xgb = study_xgb.best_trial.user_attrs.get("best_it_median", None)
print("XGB median best_it per fold:", best_it_xgb)

# тренируем на лучшиз параметрах
best_params_xgb = study_xgb.best_trial.params.copy()
final_n_estimators_xgb = int(best_it_xgb) if best_it_xgb is not None else 200
final_xgb = XGBClassifier(
    n_estimators=final_n_estimators_xgb,
    learning_rate=best_params_xgb.get("learning_rate", 0.02),
    max_depth=int(best_params_xgb.get("max_depth", 6)),
    subsample=best_params_xgb.get("subsample", 0.8),
    colsample_bytree=best_params_xgb.get("colsample_bytree", 0.8),
    gamma=best_params_xgb.get("gamma", 0.0),
    reg_alpha=best_params_xgb.get("reg_alpha", 1e-8),
    reg_lambda=best_params_xgb.get("reg_lambda", 1e-8),
    min_child_weight=int(best_params_xgb.get("min_child_weight", 1)),
    use_label_encoder=False,
    eval_metric="auc",
    random_state=42,
    n_jobs=-1
)
final_xgb.fit(X_full, y_full)

xgb_probs_test = final_xgb.predict_proba(X_test)[:, 1]
xgb_auc_test = roc_auc_score(y_test, xgb_probs_test)
print(f"Final XGB test AUC: {xgb_auc_test:.4f}")
joblib.dump(final_xgb, "models/xgb_ts_cv_best.pkl")



[I 2025-11-05 10:31:32,962] A new study created in memory with name: no-name-e9929568-0941-420d-a576-7d4640245b60


  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:33] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:34] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:35] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:38] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:42] WARNING: /w

[I 2025-11-05 10:31:46,284] Trial 0 finished with value: 0.5084823077996831 and parameters: {'learning_rate': 0.00727491708802781, 'max_depth': 10, 'subsample': 0.839196365086843, 'colsample_bytree': 0.759195090518222, 'gamma': 0.7800932022121826, 'reg_alpha': 2.5348407664333426e-07, 'reg_lambda': 3.3323645788192616e-08, 'min_child_weight': 18}. Best is trial 0 with value: 0.5084823077996831.


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:46] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:47] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:48] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:31:52] WARNING: /w

...

[I 2025-11-05 10:35:54,760] Trial 28 finished with value: 0.5214046636315005 and parameters: {'learning_rate': 0.02149598630438072, 'max_depth': 3, 'subsample': 0.9552955190515797, 'colsample_bytree': 0.9313238539551517, 'gamma': 3.325873147129732, 'reg_alpha': 0.002708751875823447, 'reg_lambda': 9.167910881718617e-08, 'min_child_weight': 18}. Best is trial 21 with value: 0.5219489065411439.


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:35:54] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:35:55] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:35:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:35:58] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:36:00] WARNING: /w

[I 2025-11-05 10:36:02,465] Trial 29 finished with value: 0.5088236439233947 and parameters: {'learning_rate': 0.008106601165418384, 'max_depth': 6, 'subsample': 0.820397026475641, 'colsample_bytree': 0.9187302215612224, 'gamma': 2.596017014795825, 'reg_alpha': 0.08762074795234935, 'reg_lambda': 7.874534359752491e-07, 'min_child_weight': 19}. Best is trial 21 with value: 0.5219489065411439.
XGB best params: {'learning_rate': 0.021093495044481346, 'max_depth': 3, 'subsample': 0.916510174102188, 'colsample_bytree': 0.77580031006743, 'gamma': 4.400366769284306, 'reg_alpha': 0.0011089224380846406, 'reg_lambda': 1.3651831039275493e-08, 'min_child_weight': 17}
XGB best CV AUC: 0.5219489065411439
XGB median best_it per fold: None
Final XGB test AUC: 0.4840


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:36:02] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


['models/xgb_ts_cv_best.pkl']

XGBoost тоже плохо работает . Видимо предсказвать цену по volume  и цене предыдущих дней попросту невозможно

In [109]:

print(f"LGBM CV AUC (best): {study_lgb.best_value:.4f} -> test AUC final: {lgb_auc_test:.4f}")
print(f"XGB  CV AUC (best): {study_xgb.best_value:.4f} -> test AUC final: {xgb_auc_test:.4f}")

LGBM CV AUC (best): 0.5605 -> test AUC final: 0.4785
XGB  CV AUC (best): 0.5219 -> test AUC final: 0.4840


Видимо предсказвать цену по volume и цене предыдущих дней попросту невозможно